In [ ]:
!pip install pennylane --quiet
!pip install pennylane-lightning --quiet
!pip install autograd --quiet

In [ ]:
import pennylane as qml
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane import numpy as np

np.random.seed(0)

In [ ]:
num_qubits = 16
num_layers = 6
weights_init = np.random.sample((num_layers, num_qubits, 1))
bias_init = np.array([0.])

In [ ]:
import tensorflow as tf
import numpy as onp
import collections

"""
Module for MNIST Digits Dataset preprocessing.
https://www.tensorflow.org/quantum/tutorials/mnist

Python 3.10.11
"""

def filter_by_classes(x, y, classes=[3,6]):
    """
    Function that filters the MNIST Digits Dataset and returns samples on 'classes'.
    Parameters:
        x: Sample images.
        y: Sample labels.
        classes: List of classes to filter.
    Returns:
        x: x filtered by 'classes'.
        y: x filtered by 'classes'.
    """
    if not all(np.isin(classes, range(0, 10))):
        return ValueError("Classes must be a list of digits (0-9).")
    x, y = x[np.isin(y, classes)], y[np.isin(y, classes)]
    if len(classes)==2:
        return x, y==classes[-1]
    else:
        return x, y

def remove_contradicting(xs, ys):

    mapping = collections.defaultdict(set)
    orig_x = {}
    # Determine the set of labels for each unique image:
    for x,y in zip(xs,ys):
       orig_x[tuple(x.flatten())] = x
       mapping[tuple(x.flatten())].add(y)

    new_x = []
    new_y = []
    for flatten_x in mapping:
      x = orig_x[flatten_x]
      labels = mapping[flatten_x]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(next(iter(labels)))
      else:
          # Throw out images that match more than one label.
          pass

    num_uniq_3 = sum(1 for value in mapping.values() if len(value) == 1 and True in value)
    num_uniq_6 = sum(1 for value in mapping.values() if len(value) == 1 and False in value)
    num_uniq_both = sum(1 for value in mapping.values() if len(value) == 2)

    print("Number of unique images:", len(mapping.values()))
    print("Number of unique 3s: ", num_uniq_3)
    print("Number of unique 6s: ", num_uniq_6)
    print("Number of unique contradicting labels (both 3 and 6): ", num_uniq_both)
    print()
    print("Initial number of images: ", len(xs))
    print("Remaining non-contradicting unique images: ", len(new_x))

    return np.array(new_x), np.array(new_y)

def preprocess_mnist_digits(classes=[3,6]):
    """"
    Function that downloads the MNIST Digits dataset with TensorFlow and performs the following tasks:
        1. Normalizes pixel values from (0, 255) to (0, 1).
        2. By default, returns only 2 classes of digits for classification (this can be deactivated or modified by the 'classes' parameter).
        3. Resizes samples to 4x4 images.
        4. Removes samples that belong to multiple classes simultaneously.
        5. Converts images to binary."
    Parameters:
    Returns:
    """

    # Download dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Rescale the images from [0,255] to the [0.0,1.0] range.
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

    # Filter to get only '3's and '6's
    x_train, y_train = filter_by_classes(x_train, y_train, classes=classes)
    x_test, y_test = filter_by_classes(x_test, y_test, classes=classes)

    print("Number of filtered training examples:", len(x_train))
    print("Number of filtered test examples:", len(x_test))

    # Resize images to 4x4
    x_train_small = tf.image.resize(x_train, (4,4)).numpy()
    x_test_small = tf.image.resize(x_test, (4,4)).numpy()

    x_train_nocon, y_train_nocon = remove_contradicting(x_train_small, y_train)

    THRESHOLD = 0.5

    # Converts non contradicting samples to binary via threshold and converting bool to float.
    x_train_bin = np.array(x_train_nocon > THRESHOLD, dtype=np.float32)
    x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

    return x_train_bin.reshape(-1, 16), y_train_nocon, x_test_bin.reshape(-1, 16), y_test



In [ ]:
X_train, Y_train, X_test, Y_test = preprocess_mnist_digits()
Y_train = np.where(Y_train==False, -1, 1)
Y_test = np.where(Y_test==False, -1, 1)

Number of filtered training examples: 12049
Number of filtered test examples: 1968
Number of unique images: 10387
Number of unique 3s:  5426
Number of unique 6s:  4912
Number of unique contradicting labels (both 3 and 6):  49

Initial number of images:  12049
Remaining non-contradicting unique images:  10338


In [ ]:
dev = qml.device("lightning.qubit", wires=range(-1, 16))

@qml.qnode(dev)
def circuit(params, data):

    qml.PauliX(wires=dev.wires[0])
    qml.BasisState(data, wires=dev.wires[1:])

    for i in range(params.shape[0]):
        for j in range(1, 16):
            if i%2 == 0:
                qml.PauliRot(params[i, j], "ZX", wires=[dev.wires[j], dev.wires[0]])
            else:
                qml.PauliRot(params[i, j], "XX", wires=[dev.wires[j], dev.wires[0]])

    qml.Hadamard(wires=dev.wires[0])
    return qml.expval(qml.PauliY(wires=dev.wires[0]))


In [ ]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [ ]:
# def square_loss(labels, predictions):
#     loss = 0
#     for l, p in zip(labels, predictions):
#         loss = loss + (l - p) ** 2

#     loss = loss / len(labels)
#     return loss

def loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += 1 - l*p

    return loss/labels.shape[0]


In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [ ]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    #return square_loss(Y, predictions)
    return loss(Y, predictions)


In [ ]:
opt = qml.AdamOptimizer(0.1)
batch_size = 10

In [ ]:
weights = weights_init
bias = bias_init

In [ ]:
for it in range(800):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (1,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # # Compute accuracy
    #predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    #acc = accuracy(Y_batch, predictions)
#square_loss(Y_train, predictions).item()
    if it%50 == 0:
        print(
            "Iter: {:5d} | Cost: {:0.5f} | Accuracy: {:0.5f} | Bias: {:0.5f}".format(
                it + 1, 1.2345, 1.2345, bias.item()
            )
        )

Iter:     1 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: -0.10000
Iter:    51 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 0.87911
Iter:   101 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 2.36135
Iter:   151 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 3.17083
Iter:   201 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 3.90810
Iter:   251 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 3.96931
Iter:   301 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 4.73566
Iter:   351 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 4.22431
Iter:   401 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 4.93861
Iter:   451 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 4.88133
Iter:   501 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 6.58969
Iter:   551 | Cost: 1.23450 | Accuracy: 1.23450 | Bias: 6.55577


KeyboardInterrupt: ignored

In [ ]:
np.save("arr.npy", weights)

In [ ]:
accuracy(Y_test, [np.sign(variational_classifier(weights, bias, x)) for x in X_test])

KeyboardInterrupt: ignored

In [ ]:
[variational_classifier(weights, bias, x) for x in X_test[:20]]

[tensor([6.24689778], requires_grad=True),
 tensor([6.24701248], requires_grad=True),
 tensor([6.24701406], requires_grad=True),
 tensor([6.24700538], requires_grad=True),
 tensor([6.24704156], requires_grad=True),
 tensor([6.24704156], requires_grad=True),
 tensor([6.24695561], requires_grad=True),
 tensor([6.24696978], requires_grad=True),
 tensor([6.24689778], requires_grad=True),
 tensor([6.24701177], requires_grad=True),
 tensor([6.24706099], requires_grad=True),
 tensor([6.24689778], requires_grad=True),
 tensor([6.24700897], requires_grad=True),
 tensor([6.24706099], requires_grad=True),
 tensor([6.24692834], requires_grad=True),
 tensor([6.24706099], requires_grad=True),
 tensor([6.24701406], requires_grad=True),
 tensor([6.24703736], requires_grad=True),
 tensor([6.24692834], requires_grad=True),
 tensor([6.24691144], requires_grad=True)]

In [ ]:
np.unique(Y_test[:20], return_counts=True)

(array([-1,  1]), array([11,  9]))